In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb
from pygam import LinearGAM

In [2]:
df = pd.read_csv('btc.csv')

In [3]:
df.head()

,Timestamp,Open,High,Low,Close,Volume,Close_7d_avg,Close_lag_1,30_day_rolling_vol,year,month,day,day_of_week,SMA_5,SMA_10,EMA_5,RSI_14
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,4334.612083,4277.932414,823.547638,2017,8,17,3,4311.910462,4362.71719,4285.080000,52.837308
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,4334.612083,4285.080000,823.547638,2017,8,18,4,4311.910462,4362.71719,4226.176667,52.837308
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,4334.612083,4108.370000,823.547638,2017,8,19,5,4311.910462,4362.71719,4197.444444,52.837308
3,2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,4334.612083,4139.980000,823.547638,2017,8,20,6,4311.910462,4362.71719,4160.392963,52.837308
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,4334.612083,4086.290000,823.547638,2017,8,21,0,4127.144000,4362.71719,4112.261975,52.837308


### TRYING DIFFERENT DIFFERENT FEATURES 

In [4]:
df = df.drop(columns=['Timestamp', 'Open', 'High', 'Low', 'SMA_5', 'SMA_10', 'EMA_5', 'RSI_14'])

# Check the result
df

,Close,Volume,Close_7d_avg,Close_lag_1,30_day_rolling_vol,year,month,day,day_of_week
0,4285.08,795.150377,4334.612083,4277.932414,823.547638,2017,8,17,3
1,4108.37,1199.888264,4334.612083,4285.080000,823.547638,2017,8,18,4
2,4139.98,381.309763,4334.612083,4108.370000,823.547638,2017,8,19,5
3,4086.29,467.083022,4334.612083,4139.980000,823.547638,2017,8,20,6
4,4016.00,691.743060,4334.612083,4086.290000,823.547638,2017,8,21,0
...,...,...,...,...,...,...,...,...,...
2715,95671.74,23368.194710,96731.161429,95780.000000,27382.555582,2025,2,18,1
2716,96644.37,16438.509540,96556.072857,95671.740000,24946.198175,2025,2,19,2
2717,98305.00,17057.391770,96798.481429,96644.370000,23983.410567,2025,2,20,3
2718,96181.98,32249.281400,96610.124286,98305.000000,24316.763529,2025,2,21,4


In [5]:
X = df[['Volume', 'Close_7d_avg', 'Close_lag_1', '30_day_rolling_vol', 'year', 'month', 'day', 'day_of_week']]
y = df['Close']

In [6]:

train_size = int(0.8 * len(df))  # 80% for training
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:] 

In [7]:
X_train

,Volume,Close_7d_avg,Close_lag_1,30_day_rolling_vol,year,month,day,day_of_week
0,795.150377,4334.612083,4277.932414,823.547638,2017,8,17,3
1,1199.888264,4334.612083,4285.080000,823.547638,2017,8,18,4
2,381.309763,4334.612083,4108.370000,823.547638,2017,8,19,5
3,467.083022,4334.612083,4139.980000,823.547638,2017,8,20,6
4,691.743060,4334.612083,4086.290000,823.547638,2017,8,21,0
...,...,...,...,...,...,...,...,...
2171,69790.657110,28396.122857,26623.410000,29791.345085,2023,8,18,4
2172,26160.753430,27920.385714,26054.000000,29412.014135,2023,8,19,5
2173,19056.912090,27475.550000,26100.010000,29251.197583,2023,8,20,6
2174,30267.242330,27003.548571,26189.990000,29771.425505,2023,8,21,0


In [8]:
X_test

,Volume,Close_7d_avg,Close_lag_1,30_day_rolling_vol,year,month,day,day_of_week
2176,44023.69978,26226.150000,26056.00,30449.724951,2023,8,23,2
2177,30205.22116,26162.812857,26432.72,30737.707396,2023,8,24,3
2178,10022.22322,26157.580000,26180.05,29940.727048,2023,8,26,5
2179,12099.64216,26157.831429,26017.37,29594.832578,2023,8,27,6
2180,22692.62655,26147.832857,26101.77,29551.466254,2023,8,28,0
...,...,...,...,...,...,...,...,...
2715,23368.19471,96731.161429,95780.00,27382.555582,2025,2,18,1
2716,16438.50954,96556.072857,95671.74,24946.198175,2025,2,19,2
2717,17057.39177,96798.481429,96644.37,23983.410567,2025,2,20,3
2718,32249.28140,96610.124286,98305.00,24316.763529,2025,2,21,4


In [9]:
##Create a Function to Evaluate Model
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

## Model Evaluation

### Overview:
We are testing and evaluating multiple regression models on a synthetic dataset, which includes models like  **Support Vector Regressor (SVR)**, **Random Forest Regressor**, **Gradient Boosting**, **XGBoost**, and others. Each model is evaluated based on its **Root Mean Squared Error (RMSE)**, **Mean Absolute Error (MAE)**, and **R² Score** for both the training and test sets.

### Models:
1. **Linear Regression** - A simple linear model.
2. **Lasso** - A linear model that adds an L1 penalty.
3. **Ridge** - A linear model with an L2 penalty.
4. **K-Neighbors Regressor** - A non-linear model that uses nearby data points for prediction.
5. **Decision Tree** - A non-linear model based on tree-based decision making.
6. **Random Forest Regressor** - An ensemble method using multiple decision trees.
7. **Adaboost Regressor** - An ensemble method that builds models sequentially.
8. **Gradient Boosting Regressor** - An ensemble method that minimizes errors sequentially.
9. **XGBoost Regressor** - An optimized gradient boosting algorithm.
10. **Support Vector Regressor (SVR)** - A regression method based on Support Vector Machines.
11. **Generalized Additive Model (GAM)** - A model that captures non-linear relationships.

### Evaluation Metrics:
- **Root Mean Squared Error (RMSE)**: Measures the average magnitude of the errors in the model's predictions.
- **Mean Absolute Error (MAE)**: Measures the average of the absolute errors between predicted and actual values.
- **R² Score**: Represents the proportion of variance in the dependent variable that is predictable from the independent variables.

### Output:
Each model's performance for both the **training** and **test** datasets is printed out, including the RMSE, MAE, and R² score. This allows for easy comparison between the models and helps identify the best-performing one for the given dataset.


In [10]:
# Define models to test
models = {
    "Random Forest Regressor": RandomForestRegressor(),
    "Gradient Boost Regressor": GradientBoostingRegressor(),
    "XGBoost Regressor": xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42),
    "Support Vector Regressor": SVR(kernel='rbf'),
    "Generalized Additive Model": LinearGAM()
}

# Function to evaluate the model performance
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

# Training, prediction, and evaluation loop
for i in range(len(models)):
    model = list(models.values())[i]
    model_name = list(models.keys())[i]

    # Train model
    if model_name == 'Support Vector Regressor':  # SVR requires scaled data
        model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
    else:
        model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

    # Evaluate model
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    # Print results for each model
    print(model_name)
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))
    print('----------------------------------')

    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))

    print('='*35)
    print('\n')

Random Forest Regressor
Model performance for Training set
- Root Mean Squared Error: 369.9157
- Mean Absolute Error: 211.1554
- R2 Score: 0.9995
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 14629.1052
- Mean Absolute Error: 7715.2518
- R2 Score: 0.5633


Gradient Boost Regressor
Model performance for Training set
- Root Mean Squared Error: 695.8919
- Mean Absolute Error: 426.6182
- R2 Score: 0.9981
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 14669.8536
- Mean Absolute Error: 7694.5496
- R2 Score: 0.5609


XGBoost Regressor
Model performance for Training set
- Root Mean Squared Error: 166.6685
- Mean Absolute Error: 117.3306
- R2 Score: 0.9999
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 15023.6479
- Mean Absolute Error: 8016.2214
- R2 Score: 0.5395


Support Vector Regressor
Model performance for Training set
- Root Mean Squared Error: 17937.

In [11]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize Random Forest model
rf = RandomForestRegressor(random_state=42)

# Apply GridSearchCV
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
grid_search_rf.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters for Random Forest:", grid_search_rf.best_params_)
print("Best score for Random Forest:", grid_search_rf.best_score_)

# Evaluate the best model
best_rf = grid_search_rf.best_estimator_
y_train_pred_rf = best_rf.predict(X_train)
y_test_pred_rf = best_rf.predict(X_test)

# Evaluate performance
model_train_mae_rf, model_train_rmse_rf, model_train_r2_rf = evaluate_model(y_train, y_train_pred_rf)
model_test_mae_rf, model_test_rmse_rf, model_test_r2_rf = evaluate_model(y_test, y_test_pred_rf)

print('Model performance for Training set (Tuned RF):')
print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse_rf))
print("- Mean Absolute Error: {:.4f}".format(model_train_mae_rf))
print("- R2 Score: {:.4f}".format(model_train_r2_rf))

print('----------------------------------')

print('Model performance for Test set (Tuned RF):')
print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse_rf))
print("- Mean Absolute Error: {:.4f}".format(model_test_mae_rf))
print("- R2 Score: {:.4f}".format(model_test_r2_rf))


Fitting 5 folds for each of 162 candidates, totalling 810 fits


KeyboardInterrupt: 